In [ ]:
import openeo
from openeo.processes import *
import geopandas as gpd

In [ ]:
# connect to the openEO service
conn = openeo.connect("openeo.cloud").authenticate_oidc()


In [ ]:
# describe the GFM collection
conn.describe_collection("GFM")

In [ ]:
# set parameters
# extract spatial extent from aoi shapefile
bbox = gpd.read_file(r"Data\Data\baringo_bbox\baringo_bbox.shp").total_bounds
spatial_extent = {
    "west": bbox[0],
    "south": bbox[1],
    "east": bbox[2],
    "north": bbox[3]}

# temporal extent from 2018-2025
temporal_extent = ["2018-02-01", "2018-03-01"]  #example 1 month


# define which collection to use
collection = "GFM"

# load the collection with the specified spatial and temporal extents
gfm_data = conn.load_collection(
    collection, 
    spatial_extent=spatial_extent, 
    temporal_extent=temporal_extent, 
    bands = ["ensemble_flood_extent"]
)

# apply a reducer to sum the flood extent over time
gfm_sum = gfm_data.reduce_dimension(dimension="t", reducer=sum)

# save the result as a GeoTIFF file with a specified tile grid
gfm_sum_tiff = gfm_sum.save_result(format="GTiff", options={"tile_grid": "wgs84-1degree"})


In [ ]:
# create and start a job to execute the process
job = gfm_sum_tiff.create_job(title = "flood_extent").start_job()

In [ ]:
# wait for the job to finish
job.status()

In [ ]:
# download the results
job.get_results().download_files("./gfm1/flood_extent_wgs/")

TESTING FOR ALL MONTHS

In [ ]:
from datetime import datetime, timedelta
import os
import time
from dateutil.relativedelta import relativedelta


In [ ]:
# Define start and end date
start_date = datetime(2018, 1, 1)
end_date = datetime(2025, 6, 30)  # To date

# Folder to save results
output_folder = "./gfm/flood_extent_wgs/"
os.makedirs(output_folder, exist_ok=True)

current_date = start_date
jobs = [] #list tos store the created jobs

# Create and start all jobs
while current_date <= end_date:
    period_start = current_date
    period_end = current_date + relativedelta(months=3) - timedelta(days=1)
    
    if period_end > end_date:
        period_end = end_date
    
    temporal_extent = [period_start.strftime("%Y-%m-%d"), period_end.strftime("%Y-%m-%d")]
    print(f"Creating job for {temporal_extent[0]} to {temporal_extent[1]}")

    gfm_dataset = conn.load_collection(
        collection,
        spatial_extent=spatial_extent,
        temporal_extent=temporal_extent,
        bands=["ensemble_flood_extent"]
    )

    gfm_sum = gfm_dataset.reduce_dimension(dimension="t", reducer=sum)
    gfm_sum_tiff = gfm_sum.save_result(format="GTiff", options={"tile_grid": "wgs84-1degree"})

    job = gfm_sum_tiff.create_job(title="flood_extent")
    job.start_job()

    jobs.append((period_start.strftime('%Y_%m'), job))
    
    current_date = period_end + timedelta(days=1)

In [ ]:
job.status()

In [ ]:
for month, job in jobs:
    print(f"Downloading results for {month}...")
    job.get_results().download_files(os.path.join(output_folder, month))

visualize the data

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray

In [ ]:
path = "./gfm_fin/flood_extent_wgs/WGS84_E35N0_20180102T155529.tif"
flood_extent = rioxarray.open_rasterio(path)
print(flood_extent)

In [ ]:
# Extract flood band and clean no-data values
flood_extent_clean = flood_extent.sel(band=1)
flood_extent_clean = flood_extent_clean.where(flood_extent_clean != -9999)


# Meshgrid for plotting
X, Y = np.meshgrid(flood_extent_clean.x.values, flood_extent_clean.y.values)

# Plot
plt.figure(figsize=(6,6))
plt.title("Flood Extent", fontsize=12)
f = plt.contourf(X, Y, flood_extent_clean, cmap='Blues')
plt.colorbar(label="Flood extent")  
plt.show()